In [1]:
# Scikit-Learn ≥0.20 is required
import sklearn

# TensorFlow ≥2.0-preview is required
import tensorflow
assert tensorflow.__version__ >= "2.0"

#import tensorflow.keras

# Common imports
import numpy as np
import os

import matplotlib.pyplot as plt


from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Input,Convolution2D,MaxPooling2D,Dense,Dropout,Activation,Flatten,Lambda,Reshape

from tensorflow.keras import backend as K

In [2]:
class DenseTranspose(tensorflow.keras.layers.Layer):
    def __init__(self, dense, activation=None, **kwargs):
        self.dense = dense
        self.activation = tensorflow.keras.activations.get(activation)
        super().__init__(**kwargs)
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name="bias",
                                      shape=[self.dense.input_shape[-1]],
                                      initializer="zeros")
        super().build(batch_input_shape)
    def call(self, inputs):
        z = inputs @ K.transpose(self.dense.weights[0]) + self.biases
        return self.activation(z)

In [3]:
class DenseCopy(tensorflow.keras.layers.Layer):
    def __init__(self, dense, activation=None, **kwargs):
        self.dense = dense
        self.activation = tensorflow.keras.activations.get(activation)
        super().__init__(**kwargs)
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name="bias",
                                      shape=[self.dense.input_shape[-1]],
                                      initializer="zeros")
        super().build(batch_input_shape)
    def call(self, inputs):
        z = inputs @ self.dense.weights[0] + self.biases
        return self.activation(z)

In [9]:
import tensorflow.keras as keras

compassos = []

Entrada = Input(shape=(128, 96,10))

for k in range(10):
    if k == 0:
        First_Crop = Lambda(lambda x: x[:,:,:,k], output_shape=(1,128,96))(Entrada)
        First_Flatten = keras.layers.Flatten()(First_Crop)
        First_Dense = Dense(10, activation="relu")(First_Flatten)
        compassos.append(First_Dense)
    else:
        Crop = Lambda(lambda x: x[:,:,:,k], output_shape=(1,128,96))(Entrada)
        Flat = keras.layers.Flatten()(Crop)
        #Densa = First_Dense(Flat)

        compassos.append(Flat)
        

outputs = [layer.output for layer in compassos[0].layers]
outputs

AttributeError: 'Tensor' object has no attribute 'layers'

In [7]:
Concatenado = tensorflow.keras.layers.concatenate([compassos[0],compassos[1]],axis=0)
Encoder_Final = keras.layers.Dense(10, activation="relu")(Concatenado)
Encoder = Model(Entrada, Encoder_Final)

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 10), (None, 12288)]

In [ ]:
Entrada_Decoder = Input(shape=(10,))
Decoder_1_Dense = DenseTranspose(Encoder_Final, activation="selu")(Entrada_Decoder)

In [ ]:
Decoders=[]
for k in range(10):
    Decoders_Crop = Lambda(lambda x: x[:,k*10:(k+1)*10-1], output_shape=(1,10))(Decoder_1_Dense)
    Decoders_Dense = keras.layers.Dense(96*128, activation="softmax")(Decoders_Crop)
    Reshape =  Reshape((128, 96,1))(Decoders_Dense)
    Decoders.append(Reshape)

In [ ]:
Saida = tensorflow.keras.layers.concatenate(Decoders,axis=3)
Decoder = Model(Entrada_Decoder, Saida)

In [ ]:
model = Sequential([Encoder,Decoder])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tensorflow.keras.optimizers.SGD(lr=0.1),
              metrics=["accuracy"])
model.build([])
model.summary()